#  Code Kata - Data

Open this notebook in MyBinder: [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/rwcitek/MyBinder.demo/main?urlpath=lab%2Ftree%2FInitial.Data.Analysis%2Fcode.kata.data-munging.ipynb)

http://codekata.com/kata/kata04-data-munging/

In weather.dat you’ll find daily weather data for Morristown, NJ for June 2002. Download this text file, then write a program to output the day number (column one) with the smallest temperature spread (the maximum temperature is the second column, the minimum the third column).


## Part One: Weather Data


In [ ]:
curl -s http://codekata.com/data/04/weather.dat > weather.dat

In [ ]:
cat --help

In [ ]:
echo -e 'hello\r\nworld\tfoobar' | cat -vetn


In [ ]:
cat -v -e -t -n weather.dat

In [ ]:
cat weather.dat |
cut -d$'\n' -f 3-32 |
tr -s ' ' '\t' |
cut -f2-4  |
tr -d '*' | 
while read day max min ; do echo $day $(( max - min )) $max $min ; done |
tee /dev/stderr |
sort -k2,2n |
head -1



In [ ]:
cat weather.dat |
cut -d$'\n' -f 3-32 |
tr -s ' ' '\t' |
cut -f2-4 |
tr -d '*' |
while read day max min ; do echo $day $(( max - min )) $max $min ; done |
tee /dev/stderr |
sort -k2,2n |
head -1


In [ ]:
head weather.dat |
awk -v min=1000 '
    BEGIN { print "Greetings!" } ;
    $1 ~ /[0-9]/ {
        diff = $2 - $3 ;
        if ( diff < min ) {
            min = diff ;
            day = $1 ;
        } ;
        print "Did if"
    } ;
    { print "hey!" ;
      print min, day, $0 ;
    }
    END {
        print min, day ;
    }
'


In [ ]:
cat weather.dat |
awk -v min=1000 '
$1 ~ /[0-9]/ {
    diff = $2-$3 ; diff = diff < 0 ? -diff : diff ;
    if (diff < min ) {
        min=diff ;
        day=$1 ;
    }
} ;
END {
    print day, min
}
'

## Part Two: Soccer League Table

The file football.dat contains the results from the English Premier League for 2001/2. The columns labeled ‘F’ and ‘A’ contain the total number of goals scored for and against each team in that season (so Arsenal scored 79 goals against opponents, and had 36 goals scored against them). Write a program to print the name of the team with the smallest difference in ‘for’ and ‘against’ goals.


In [ ]:
curl -s http://codekata.com/data/04/football.dat > football.dat 

In [ ]:
wc -l football.dat

In [ ]:
cat -vetn football.dat

In [ ]:
cat football.dat |
awk -v min=1000 '
$1 ~ /[0-9]/ {
    diff = ($7-$9) ; diff = diff < 0 ? -diff : diff ;
    if (diff < min ) {
        min=diff ; key=$2 ;
    }
} ;
END { print key, min } ;
'

## Part Three: DRY Fusion

Take the two programs written previously and factor out as much common code as possible, leaving you with two smaller programs and some kind of shared functionality.

In [ ]:
cat weather.dat |
awk -v high=2 -v low=3 -v vkey=1 -v vid=1 -v min=1000 '$vid ~ /[0-9]/ { diff = $high-$low ; if (diff < min ) { min=diff ; key=$vkey ; } } ;
                                                       END            { print key, min } '

In [ ]:
cat football.dat |
awk -v high=7 -v low=9 -v vkey=2 -v vid=1 -v min=1000 '$vid ~ /[0-9]/ { diff = $high-$low ; if (diff < min ) { min=diff ; key=$vkey ; } } ;
                                                       END            { print key, min } '

In [ ]:
cat <<'eof' > code.awk
$vid ~ /[0-9]/ { diff = $vhigh-$vlow ; if (diff < min ) { min=diff ; key=$vkey ; } } ;
END            { print key, min }
eof

< weather.dat  awk -v vhigh=2 -v vlow=3 -v vkey=1 -v vid=1 -v min=1000 -f code.awk
< football.dat awk -v vhigh=7 -v vlow=9 -v vkey=2 -v vid=1 -v min=1000 -f code.awk


In [ ]:
cat <<'eof' > code.awk
$1 ~ /[0-9]/ { diff = $vhigh-$vlow ; diff = diff < 0 ? -diff : diff ; if (diff < min ) { min=diff ; key=$vkey ; } } ;
END          { print key, min }
eof

< weather.dat  awk -v vhigh=2 -v vlow=3 -v vkey=1 -v min=1000 -f code.awk
< football.dat awk -v vhigh=7 -v vlow=9 -v vkey=2 -v min=1000 -f code.awk
